In [2]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import scipy
import scipy.stats

In [3]:
class MedianPolish:
    """Fits an additive model using Tukey's median polish algorithm"""

    #note that the self argument is default that allows reference to an instance of the
    #class that has been defined by the user
    #no argument is actally specified to be passed to the self argument by the user;
    #the first user specified argument
    #goes into the first argument listed after self in the code of the class
    def __init__(self, array): 
        """Get numeric data from numpy ndarray to self.tbl, keep the original copy in tbl_org"""
        #checks if the argument called array is actually a numerical array of the
        #object type np.ndarray by using the comparison function isinstance
        if isinstance(array, np.ndarray):
            self.tbl_org = array
            #tbl_org will allow you to compare the final polished matrix with row and column
            #effects removed to the original unpolished matrix
            self.tbl = self.tbl_org.copy()
        else:
            raise TypeError('Expected the argument to be a numpy.ndarray.')

    @staticmethod
    def csv_to_ndarray(fname): 
        """ Utility method for loading ndarray from .csv file""" 
        try:
            #generates an array-like object of type np.ndarray
            #from a comma separated values file
            return np.genfromtxt(fname, delimiter=",")	
        except Exception, e:
            print "Error loading file %s:" % fname
            raise

    def median_polish(self, max_iterations, method):
        """
            Implements Tukey's median polish alghoritm for additive models
            method - default is median, alternative is mean. That would give us result equal ANOVA.
        """
        
        grand_effect = 0
        median_row_effects = 0
        median_col_effects = 0
        #defines a vector that stores the row_effects during each iteration;
        #the vector is initialized with zeros and length equal to the number of rows
        #as determined by the function shape[0]
        row_effects = np.zeros(shape=self.tbl.shape[0])
        col_effects = np.zeros(shape=self.tbl.shape[1])

        for i in range(max_iterations):
            if method == 'median':
                #note that np.median's second arg specifies along which axis to perform
                #the median calculation, with axis = 1 being rows, and axis = 0 being columns
                row_medians = np.median(self.tbl, 1)
                #the next line keeps a running total of the row effects that have been
                #subtracted out during the iterative polishing procedure
                row_effects += row_medians
                median_row_effects = np.median(row_effects)
            elif method == 'average':
                row_medians = np.average(self.tbl, 1) 
                row_effects += row_medians
                median_row_effects = np.average(row_effects)
            
            #not sure what the grand_effect variable is keeping track of; it is
            #returned but never used in later calculations
            grand_effect += median_row_effects
            
            #the following line I have commented out, don't know why it is needed
            #row_effects -= median_row_effects
            
            #the following line reshapes the the row_medians array into a column
            #this reshaping is necessary because each element in row_medians
            #is the median from each row, which needs to be subtracted from each row
            #by reshaping into a column, can simply subtract this column from each
            #column of the data matrix, self.tbl
            #the np.newaxis function adds a new dimension, such that now the array
            #is a matrix with n number of rows and 1 column; hence the array is now a
            #column vector.
            self.tbl -= row_medians[:, np.newaxis]

            if method == 'median':
                col_medians = np.median(self.tbl, 0) 
                col_effects += col_medians
                median_col_effects = np.median(col_effects)
            elif method == 'average':
                col_medians = np.average(self.tbl, 0) 
                col_effects += col_medians
                median_col_effects = np.average(col_effects)

            #note that by default, arrays such as col_medians are formulated as a row vector
            #for use in numerical calculations
            self.tbl -= col_medians
            
            #the following line I have commented out, don't know why it is needed
            #col_effects -= col_medians
            
            grand_effect += median_col_effects

        return grand_effect, col_effects, row_effects , self.tbl, self.tbl_org


if __name__ == "__main__":

    #makes output more legible for debugging
    np.set_printoptions(precision = 5, suppress = True)
    
    data_file = open('/Users/markfang/Dropbox/UCSD Grad work/RNA-Yeo Lab/Python scripts/150126 smNPC LOPAC Prestwick.csv', 'rU')
    data_table = np.genfromtxt(data_file, delimiter=',')
    
    #the actual data is listed in rows 5 to 388, and columns 2 through 17; the rest are
    #data table label strings
    firstrow = 4
    lastrow = 388
    firstcolumn = 1
    lastcolumn = 17
    raw_data = data_table[firstrow:lastrow, firstcolumn:lastcolumn]

    num_rows = raw_data.shape[0]
    num_col = raw_data.shape[1]
    
    #the data table needs to be reshaped into 16 plates with 16 rows and 24 columns
    #this way I can run the polish on each of the 16 plates.  the next few lines
    #accomplish this reshaping
    raw_data = np.reshape(raw_data, (1, num_rows * num_col), order = 'F')

    #16 plates, 16 rows, 24 columns
    plates = 16
    plate_rows = 16
    plate_cols = 24
    raw_data = np.reshape(raw_data, (plates, plate_rows, plate_cols), order = 'C')

    bscores = []
    
    #calculate b scores on a plate-by-plate basis
    for i in xrange(0, plates):
    
        #iterate over each plate and perform the median polish
        arr = raw_data[i, :, :]

        tbl_avg = np.average(arr)
        #subtract out the average for each plate, thus normalizing out plate effects
        arr -= tbl_avg
        #pass each plate's data matrix into the MedianPolish object
        mp = MedianPolish(arr)

        #first argument indicates number of iterations to be run
        #ce is an ndarray storing the column effects after n iterations of polishing
        #re is an ndarray storing the row effects after n iterations of polishing
        #resid is the data table that has been polished to remove 
        ge, ce, re, resid, tbl_org =  mp.median_polish(10, "median") 

        re_reshape = re[:, np.newaxis]
        
        #the tbl_org returned by mmp.median_polish has had the tbl_avg subtracted,
        #so to get the initial data table back need to add the tbl_avg back
        tbl_org += tbl_avg

        #the next few lines compute the median absolute deviation
        #MAD = median(|x - median(x)|)
        tbl_resid_minusmedians = resid - np.median(resid)
        median_absdev = np.median(np.absolute(tbl_resid_minusmedians))
        
        #find the b scores of the plate
        tbl_bscore = resid / median_absdev
        
        #collect the b scores of each plate into the array bscores
        bscores.append(tbl_bscore)
    
    print(bscores)

[array([[-16.56013,   0.73117,   0.15096,  -0.15579,  -2.04506,  -1.07335,
         -2.15263,  -1.84015,  -1.64742,   1.61433,   3.13974,   4.01776,
          4.27345,   3.65749,   4.98997,   1.08293,  -2.37005,  -4.13129,
         -2.54045,   1.28088,  -0.93788,   1.61402,  -0.64904,   0.51909],
       [ -1.57703,  -0.02929,  -2.23651,  15.63675,  -1.00069,  -0.24223,
          0.46784,  -2.13381,   0.4653 ,  -0.69722,   2.44106,   1.67323,
          1.07731,  -0.12704,   0.63694,   1.40575,  -1.65942,  18.33856,
          0.04144,  -1.54956,   0.49564,   0.91446,  -4.08537,  -1.23564],
       [ -0.04429,  -0.60775,   0.12634,   1.22221,  -0.30412,  -0.06963,
          1.92586,   0.20309,  -3.28045,   0.04554,   0.37563,   1.1433 ,
          1.19907,   1.62627,  -0.10731,   0.78096,  -0.81132,  -1.86256,
         -0.38812,   0.23566,  -1.99163,   2.0392 ,  -0.20531,  -2.44793],
       [  1.21715,  -0.1168 ,   1.4693 ,   0.85621,   1.88635,   0.19777,
         -0.25176,  -0.19086,   0.

In [11]:
    bscores_median = np.median(bscores)

    bscores_copy = bscores
    #subtract the overall median; this is later used to calculate the MAD
    for i in range(0, plates):
        for j in range(0, plate_rows):
            for k in range(0, plate_cols):
                bscores_copy[i][j][k] -= bscores_median

    #1.4826 is scaling constant to make 1 MAD comparable in magnitude to 1 SD
    #see (chung, strulovici et al 2007)
    mad = 1.4826 * np.median(np.absolute(bscores_copy))
    
    #2 MAD corresponds to false positive rate of 0.023 under a normal distribution
    threshold = 2
    upper_threshold = bscores_median + threshold * mad
    lower_threshold = bscores_median - threshold * mad

    hits_mad = []
    sg_enhancers_mad = []
    
    unique_plates = plates / 2

    for i in range(0, unique_plates):
        for j in range(0, plate_rows):
            for k in range(0, plate_cols):
                #find median of replicates, then check if it is +/- 3 MAD
                median_ofrepl = np.median([bscores[i][j][k], bscores[i + unique_plates][j][k]])
                if median_ofrepl < lower_threshold:
                    #collect plate, row, column coordinates of hits
                    #indexed from 1 rather than 0
                    hits_mad.append(i + 1)
                    hits_mad.append(j + 1)
                    hits_mad.append(k + 1)
                if median_ofrepl > upper_threshold:
                    sg_enhancers_mad.append(i + 1)
                    sg_enhancers_mad.append(j + 1)
                    sg_enhancers_mad.append(k + 1)


    len_hits_mad = len(hits_mad)
    #reshape into ordered triples giving the plate, row, and column coordinates for hits
    hits_mad = np.reshape(hits_mad, (len_hits_mad / 3, 3))
    len_sg_enhancers_mad = len(sg_enhancers_mad)
    sg_enhancers_mad = np.reshape(sg_enhancers_mad, (len_sg_enhancers_mad / 3, 3))


[[ 1  1  1]
 [ 1 12 24]
 [ 1 14  1]
 [ 1 14  2]
 [ 1 15 24]
 [ 1 16  1]
 [ 1 16  2]
 [ 1 16  3]
 [ 1 16 24]
 [ 2  1  1]
 [ 2  2  1]
 [ 2  2  3]
 [ 2  3  6]
 [ 2  3 17]
 [ 2 14 24]
 [ 2 15  3]
 [ 2 15  4]
 [ 2 15 21]
 [ 2 15 23]
 [ 2 15 24]
 [ 2 16  1]
 [ 2 16  3]
 [ 3  2 22]
 [ 3  2 24]
 [ 3  9  3]
 [ 3 10 16]
 [ 3 14  1]
 [ 3 16  1]
 [ 3 16  5]
 [ 3 16  7]
 [ 4  1  1]
 [ 4  1 24]
 [ 4  2  1]
 [ 5  2  3]
 [ 5  2  4]
 [ 5  8  3]
 [ 5 15  6]
 [ 5 15 22]
 [ 5 16 16]
 [ 5 16 22]
 [ 6  2  3]
 [ 6  3 19]
 [ 6 14  2]
 [ 6 15  2]
 [ 6 15  4]
 [ 7  2  1]
 [ 7  2  3]
 [ 7 11  1]
 [ 7 16 23]
 [ 8  2  3]
 [ 8  3 22]
 [ 8  7 15]
 [ 8 10 15]
 [ 8 13 24]
 [ 8 16  3]]


In [6]:
variances = []
averages = []

for i in range(0, unique_plates):
    for j in range (0, plate_rows):
        for k in range (0, plate_cols):
            first_repl = bscores[i][j][k]
            second_repl = bscores[i + unique_plates][j][k]
            sample_avg = (first_repl + second_repl) / 2
            sample_variance = ((first_repl - sample_avg) ** 2 + (second_repl - sample_avg) ** 2) / 1
            variances.append(sample_variance)
            averages.append(sample_avg)

#according to paper wright and simon 2003, the sample variances multipled by two constants
#a and b follow an F distribution with parameters (n-k) and 2a, where n is number of
#replicates, k is number of group (experimental, control, etc)
#in my data, I have duplicates and 1 group, so n = 2, k = 1.
param = scipy.stats.f.fit(variances)

#after fitting we want to find the value of a and b, since these are the parameters for
#the putative inverse gamma distribution that is the true distribution of the variances
#of the small molecule screen.  finding a and b will help us specify the inverse
#gamma distribution, which will improve the power of our t tests (wright and simon 2003)
#find parameter a: since the fitted distribution has parameters (n-k) and 2a, we can
#take the second parameter and divide by 2 to get a
invgammaparam_a = param[1] / 2

#we fit an F distribution to our variances, and we see that the scaling s is stored in the
#fourth parameter.  a*b*variances fits to an F distribution with area under the curve = 1
#since F is a probability distribution (scaling = 1)
#thus when we simply fit our variances to an F distribution,
#we may get a scaling s =/= 1 (area under the curve not equal 1)
#since multiplying variates by a constant changes the scaling of the fitted F distribution
#we can figure out what a*b is by knowing that multiplying the variances
#by a*b brings the scaling up to 1; hence a*b equals the multiplicative inverse
#of the current scaling.  from here we can find b because we already have a
invgammaparam_b = (1 / param[3]) / invgammaparam_a

(1.0924273676483396, 1.6643627869302637, 5.4743155355404714e-09, 0.89416098197061511)
0.832181393465
1.34389792645


In [12]:
#convert lists to numpy objects ndarrays to be able to easily perform math operations
variances = np.asarray(variances)
averages = np.asarray(averages)

#variances that have been fitted to the inverse gamma distribution
rvm_variances = ((2 - 1) * variances + 2 * invgammaparam_a * (1 / (invgammaparam_a * invgammaparam_b)))/((2 - 1) + 2 * invgammaparam_a)          

#calculate std dev from the variances that have been fitted to the inverse gamma distribution
denominator = np.sqrt(rvm_variances * 1 / 2)

#calculate t statistic for each compound, using the rvm_variances
t_stats = (averages - 0) / denominator

len_t_stats = t_stats.shape[0]

p_val = []
df = 2 * invgammaparam_a

#p values for 2 tailed t tests
for i in range(0, len_t_stats):
    if t_stats[i] <= 0:
        prob = scipy.stats.t.cdf(t_stats[i], df)
        prob *= 2
        p_val.append(prob)
    else:
        prob = scipy.stats.t.sf(t_stats[i], df)
        prob *= 2
        p_val.append(prob)

p_val = np.asarray(p_val)
p_val = p_val[:, np.newaxis]

coordinates = []

#list out the plate, row, and col coordinates to be concatenated with the p values
#this helps keep track of where each p value came from in the physical location
#on the plates after the p values are sorted in order to do FDR controlling
#such as benjamini hochberg
#these are indexed from 1 not 0 for ease of interpretation (i.e. plate 1 rather than plate 0)
for i in range(0, unique_plates):
    for j in range(0, plate_rows):
        for k in range(0, plate_cols):
            coordinates.append(i + 1)
            coordinates.append(j + 1)
            coordinates.append(k + 1)

coordinates = np.asarray(coordinates)
len_coord = len(coordinates)
coordinates = np.reshape(coordinates, (len_coord / 3, 3), order = 'C')

p_val_coord = np.concatenate((p_val, coordinates), axis = 1)
p_val_coord = np.ndarray.tolist(p_val_coord)

p_val_coord.sort()

[0.000898773830858683, 2.0, 8.0, 22.0]
(150.69413427582421, 142.89469666516808)


In [13]:
#ignoring multiple hypothesis testing and therefore not controlling for FDR
sigma = 0.05
hits_t_test = []

len_p_val_coord = len(p_val_coord)

for i in range(0, len_p_val_coord):
    if p_val_coord[i][0] < sigma:
        hits_t_test.append(p_val_coord[i])
print(len(hits_t_test))        
print(hits_t_test)


75
[[0.000898773830858683, 2.0, 8.0, 22.0], [0.002958411282438368, 7.0, 7.0, 18.0], [0.00525768542696488, 6.0, 12.0, 16.0], [0.012114747689088701, 4.0, 13.0, 18.0], [0.012648714806241848, 2.0, 14.0, 12.0], [0.014621585625539533, 6.0, 15.0, 2.0], [0.014740249163014698, 4.0, 12.0, 17.0], [0.018798410445199337, 3.0, 7.0, 6.0], [0.02072246923508463, 3.0, 2.0, 22.0], [0.021108759548365576, 6.0, 10.0, 24.0], [0.02179414771642985, 6.0, 15.0, 4.0], [0.02295140117362713, 5.0, 2.0, 11.0], [0.022973621655771603, 7.0, 6.0, 23.0], [0.025603571240974246, 4.0, 1.0, 15.0], [0.026271369369264636, 4.0, 11.0, 1.0], [0.02774044665000746, 3.0, 16.0, 1.0], [0.02815084566578395, 8.0, 2.0, 15.0], [0.028418841992675446, 2.0, 14.0, 14.0], [0.028992834128322077, 1.0, 2.0, 4.0], [0.02900348517173377, 7.0, 1.0, 20.0], [0.029284358358202187, 5.0, 5.0, 23.0], [0.029765822176490597, 2.0, 2.0, 1.0], [0.030051160832453728, 8.0, 1.0, 19.0], [0.030138757668333026, 3.0, 12.0, 24.0], [0.030526009923099395, 2.0, 6.0, 24.0],

In [41]:
#benjamini hochberg method to control FDR
fdr = 0.05
k = 1.0
i = 0
m = unique_plates * plate_rows * plate_cols

hits_fdr = []

print(k / m * fdr)

while p_val_coord[i][0] < (k / m * fdr):
    hit = p_val_coord[i]
    hits_fdr.append(hit)
    i += 1
    k += 1.0
    
print(hits_fdr)

1.62760416667e-05
[]
